In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import os
import itertools
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams.update({'font.size': 20, 'text.usetex': True})
import seaborn as sns
import pickle
import torch
import dataframe_image as dfi
from tqdm import tqdm

In [2]:
model_folder = '../Data/Landscape_Models/'

grid_space = {
    "model_embedding": [
        "linear-one_hot",
        "cnn-one_hot",
        "fnn-tape",
        "fnn-esm1b",
        "fnn-esm1v",
        "fnn-esm2_8m",
        "fnn-esm2_35m",
        "fnn-esm2_150m",
        "fnn-esm2_650m",
        "fnn-esm2",
        "fnn-prottrans",
        "fnn-carp_600k",
        "fnn-carp_38M",
        "fnn-carp_76M",
        "fnn-carp",
        "cnn-tape_all",
        "cnn-esm1b_all",
        "cnn-esm1v_all",
        "cnn-esm2_8m_all",
        "cnn-esm2_35m_all",
        "cnn-esm2_150m_all",
        "cnn-esm2_650m_all",
        "cnn-esm2_all",
        "cnn-prottrans_all",
        "cnn-carp_600k_all",
        "cnn-carp_38M_all",
        "cnn-carp_76M_all",
        "cnn-carp_all",
        "linear-augmented_esm1v",
        "linear-augmented_esm2_8m",
        "linear-augmented_esm2_35m",
        "linear-augmented_esm2_150m",
        "linear-augmented_esm2_650m",
        "linear-augmented_esm2",
        "linear-augmented_carp_600k",
        "linear-augmented_carp_38M",
        "linear-augmented_carp_76M",
        "linear-augmented_carp",
        "linear-augmented_evcoupling",
    ],
    "split": ["one_vs_rest", "two_vs_rest", "three_vs_rest", "low_vs_high"],
    "noise": [True, False],
    "iteration": [0, 1, 2]
}

def grid_space_iter(grid):
    iter_values = list(itertools.product(*grid.values()))
    combos = [dict(zip(grid.keys(), v)) for v in iter_values]
    output = []
    for combo in combos:
        combo["model"] = combo["model_embedding"].split("-")[0]
        combo["embedding"] = combo["model_embedding"].split("-")[1]
        output.append(combo)
        combo["num_gpus"] = 1
        if combo["embedding"] == "esm2":
            combo["num_gpus"] = 2
    return output

grid_space = grid_space_iter(grid_space)

def output_str(model_combo):
    return f"{model_combo['model_embedding']}_{model_combo['split']}_{str(model_combo['noise'])}_iter{model_combo['iteration']}"

In [5]:
models = []
embeddings = []
model_embeddings = []
model_embedding_labels = []
model_types = []
noises = []
splits = []
noise_splits = []
iterations = []
y_preds = []
y_trues = []
y_vars = []
spearman_rs = []
weighted_mses = []
num_params = []
model_embedding_labels_dict = {
    'linear-one_hot': 'Linear', 
    'cnn-one_hot': 'CNN', 
    'fnn-tape': 'TAPE', 
    'fnn-esm1b': 'ESM-1b', 
    'fnn-esm1v': 'ESM-1v', 
    'fnn-esm2_8m': 'ESM-2 (8M)', 
    'fnn-esm2_35m': 'ESM-2 (35M)', 
    'fnn-esm2_150m': 'ESM-2 (150M)', 
    'fnn-esm2_650m': 'ESM-2 (650M)', 
    'fnn-esm2': 'ESM-2', 
    'fnn-prottrans': 'ProtT5',
    'fnn-carp_600k': 'CARP (600k)',
    'fnn-carp_38M': 'CARP (38M)',
    'fnn-carp_76M': 'CARP (76M)',
    'fnn-carp': 'CARP', 
    'cnn-tape_all': 'TAPE (CNN)', 
    'cnn-esm1b_all': 'ESM-1b (CNN)', 
    'cnn-esm1v_all': 'ESM-1v (CNN)', 
    'cnn-esm2_8m_all': 'ESM-2 (8M, CNN)',
    'cnn-esm2_35m_all': 'ESM-2 (35M, CNN)',
    'cnn-esm2_150m_all': 'ESM-2 (150M, CNN)',
    'cnn-esm2_650m_all': 'ESM-2 (650M, CNN)',
    'cnn-esm2_all': 'ESM-2 (CNN)', 
    'cnn-prottrans_all': 'ProtT5 (CNN)', 
    'cnn-carp_600k_all': 'CARP (600k, CNN)',
    'cnn-carp_38M_all': 'CARP (38M, CNN)',
    'cnn-carp_76M_all': 'CARP (76M, CNN)',
    'cnn-carp_all': 'CARP (CNN)', 
    'linear-augmented_esm1v': 'ESM-1v (Augmented)', 
    'linear-augmented_esm2_8m': 'ESM-2 (8M, Augmented)',
    'linear-augmented_esm2_35m': 'ESM-2 (35M, Augmented)',
    'linear-augmented_esm2_150m': 'ESM-2 (150M, Augmented)',
    'linear-augmented_esm2_650m': 'ESM-2 (650M, Augmented)',
    'linear-augmented_esm2': 'ESM-2 (Augmented)', 
    'linear-augmented_carp_600k': 'CARP (600k, Augmented)',
    'linear-augmented_carp_38M': 'CARP (38M, Augmented)',
    'linear-augmented_carp_76M': 'CARP (76M, Augmented)',
    'linear-augmented_carp': 'CARP (Augmented)', 
    'linear-augmented_evcoupling': 'EVMutation (Augmented)'
}
model_type_dict = {
    'linear-one_hot': 'Baseline', 
    'cnn-one_hot': 'CNN', 
    'fnn-tape': 'FNN', 
    'fnn-esm1b': 'FNN', 
    'fnn-esm1v': 'FNN',
    'fnn-esm2_8m': 'FNN',
    'fnn-esm2_35m': 'FNN',
    'fnn-esm2_150m': 'FNN',
    'fnn-esm2_650m': 'FNN',
    'fnn-esm2': 'FNN', 
    'fnn-prottrans': 'FNN', 
    'fnn-carp_600k': 'FNN',
    'fnn-carp_38M': 'FNN',
    'fnn-carp_76M': 'FNN',
    'fnn-carp': 'FNN', 
    'cnn-tape_all': 'CNN', 
    'cnn-esm1b_all': 'CNN', 
    'cnn-esm1v_all': 'CNN',
    'cnn-esm2_8m_all': 'CNN',
    'cnn-esm2_35m_all': 'CNN',
    'cnn-esm2_150m_all': 'CNN',
    'cnn-esm2_650m_all': 'CNN',
    'cnn-esm2_all': 'CNN', 
    'cnn-prottrans_all': 'CNN',
    'cnn-carp_600k_all': 'CNN',
    'cnn-carp_38M_all': 'CNN',
    'cnn-carp_76M_all': 'CNN',
    'cnn-carp_all': 'CNN', 
    'linear-augmented_esm1v': 'Augmented',
    'linear-augmented_esm2_8m': 'Augmented',
    'linear-augmented_esm2_35m': 'Augmented',
    'linear-augmented_esm2_150m': 'Augmented',
    'linear-augmented_esm2_650m': 'Augmented',
    'linear-augmented_esm2': 'Augmented', 
    'linear-augmented_carp_600k': 'Augmented',
    'linear-augmented_carp_38M': 'Augmented',
    'linear-augmented_carp_76M': 'Augmented',
    'linear-augmented_carp': 'Augmented', 
    'linear-augmented_evcoupling': 'Augmented'
}
num_params_dict = {
    'linear-one_hot': 800,
    'cnn-one_hot': 800, 
    'fnn-tape': 9.1e7, 
    'fnn-esm1b': 6.5e8, 
    'fnn-esm1v': 6.5e8, 
    'fnn-esm2_8m': 8e6, 
    'fnn-esm2_35m': 3.5e7, 
    'fnn-esm2_150m': 1.5e8, 
    'fnn-esm2_650m': 6.5e8, 
    'fnn-esm2': 3e9, 
    'fnn-prottrans': 3e9,
    'fnn-carp_600k': 6e5,
    'fnn-carp_38M': 3.8e7,
    'fnn-carp_76M': 7.6e7,
    'fnn-carp': 6.4e8, 
    'cnn-tape_all': 9.1e7, 
    'cnn-esm1b_all': 6.5e8, 
    'cnn-esm1v_all': 6.5e8, 
    'cnn-esm2_8m_all': 8e6,
    'cnn-esm2_35m_all': 3.5e7,
    'cnn-esm2_150m_all': 1.5e8,
    'cnn-esm2_650m_all': 6.5e8,
    'cnn-esm2_all': 3e9, 
    'cnn-prottrans_all': 3e9, 
    'cnn-carp_600k_all': 6e5,
    'cnn-carp_38M_all': 3.8e7,
    'cnn-carp_76M_all': 7.6e7,
    'cnn-carp_all': 6.4e8, 
    'linear-augmented_esm1v': 6.5e8, 
    'linear-augmented_esm2_8m': 8e6,
    'linear-augmented_esm2_35m': 3.5e7,
    'linear-augmented_esm2_150m': 1.5e8,
    'linear-augmented_esm2_650m': 6.5e8,
    'linear-augmented_esm2': 3e9, 
    'linear-augmented_carp_600k': 6e5,
    'linear-augmented_carp_38M': 3.8e7,
    'linear-augmented_carp_76M': 7.6e7,
    'linear-augmented_carp': 6.4e8, 
    'linear-augmented_evcoupling': 800
}
split_labels_dict = {'low_vs_high': 'Low vs High', 'one_vs_rest': 'One vs Rest', 'two_vs_rest': 'Two vs Rest', 'three_vs_rest': 'Three vs Rest'}

def noise_split_converter(noise, split):
    if noise:
        return split_labels_dict[split] + " FLIGHTED"
    return split_labels_dict[split]

for model_combo in tqdm(grid_space):
    models += [model_combo['model']]
    embeddings += [model_combo['embedding']]
    model_embeddings += [model_combo['model_embedding']]
    splits += [model_combo['split']]
    noises += [model_combo['noise']]
    noise_splits += [noise_split_converter(model_combo['noise'], model_combo['split'])]
    iterations += [model_combo['iteration']]
    model_embedding_labels += [model_embedding_labels_dict[model_combo['model_embedding']]]
    model_types += [model_type_dict[model_combo['model_embedding']]]
    num_params += [num_params_dict[model_combo['model_embedding']]]

    output_dir = os.path.join(model_folder, output_str(model_combo))
    test_pred = np.load(os.path.join(output_dir, 'test_preds.npy'))
    test_true = np.load(os.path.join(output_dir, 'test_trues.npy'))
    test_var = np.load(os.path.join(output_dir, 'test_vars.npy'))
    test_sequences = np.load(os.path.join(output_dir, 'test_sequences.npy'))
    y_preds += [test_pred]
    y_trues += [test_true]
    y_vars += [test_var]
    spearman_rs += [stats.spearmanr(test_pred, test_true).correlation]
    if model_combo['noise']:
        weighted_mses += [np.mean((test_true - test_pred)**2 * 1/(test_var))]
    else:
        weighted_mses += [mean_squared_error(test_true, test_pred)]

100%|█████████████████████████████████████████████████████████████| 936/936 [00:13<00:00, 68.36it/s]


In [6]:
results_df = {'Model': model_embedding_labels, 'Model Type': model_types, 'Task': noise_splits, 'Iteration': iterations, 'Spearman R': spearman_rs, 'Weighted MSE': weighted_mses, 'Number of Parameters': num_params}
results_df = pd.DataFrame(results_df)

In [8]:
def p_value_min(s, props=''):
    means = s.apply(lambda x: float(x.split('±')[0])).values
    stds = s.apply(lambda x: float(x.split('±')[1])).values
    pvalues = [stats.ttest_ind_from_stats(mean, std, 3, np.min(means), stds[np.argmin(means)], 3, equal_var=False).pvalue for (mean, std) in zip(means, stds)]
    return np.where(np.array(pvalues) > 0.05, props, '')

def highlight_min(s, props=''):
    means = s.apply(lambda x: float(x.split('±')[0])).values
    return np.where(means == np.min(means), props, '')


row_order = ['Linear', 'TAPE', 'ESM-1b', 'ESM-1v', 'ESM-2 (8M)', 'ESM-2 (35M)', 'ESM-2 (150M)', 'ESM-2 (650M)', 'ESM-2', 'ProtT5', 'CARP (600k)', 'CARP (38M)', 'CARP (76M)', 'CARP', 'CNN', 'TAPE (CNN)', 'ESM-1b (CNN)', 'ESM-1v (CNN)', 'ESM-2 (8M, CNN)', 'ESM-2 (35M, CNN)', 'ESM-2 (150M, CNN)', 'ESM-2 (650M, CNN)', 'ESM-2 (CNN)', 'ProtT5 (CNN)', 'CARP (600k, CNN)', 'CARP (38M, CNN)', 'CARP (76M, CNN)', 'CARP (CNN)', 'ESM-1v (Augmented)', 'ESM-2 (8M, Augmented)', 'ESM-2 (35M, Augmented)', 'ESM-2 (150M, Augmented)', 'ESM-2 (650M, Augmented)', 'ESM-2 (Augmented)', 'CARP (600k, Augmented)', 'CARP (38M, Augmented)', 'CARP (76M, Augmented)', 'CARP (Augmented)', 'EVMutation (Augmented)']
column_order = ['One vs Rest', 'One vs Rest FLIGHTED', 'Two vs Rest', 'Two vs Rest FLIGHTED', 'Three vs Rest', 'Three vs Rest FLIGHTED', 'Low vs High', 'Low vs High FLIGHTED']
results_df_mean_std = results_df.groupby(['Model', 'Task']).agg(['mean', 'std'])['Weighted MSE']
results_df_mean_std_grouped = results_df_mean_std.apply(lambda x: f"{x['mean']:.4f} ± {x['std']:.4f}", axis=1)
results_df_combined = results_df_mean_std_grouped.to_frame(name='Weighted MSE').reset_index().pivot(values='Weighted MSE', index='Model', columns='Task').reindex(row_order)[column_order]
results_df_combined_styled = results_df_combined.style.apply(p_value_min, props='background-color:yellow', axis=0).apply(highlight_min, props='font-weight:bold', axis=0)
results_df_combined_styled

/tmp/ipykernel_520159/2627982402.py:14: FutureWarning: ['Model Type'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  results_df_mean_std = results_df.groupby(['Model', 'Task']).agg(['mean', 'std'])['Weighted MSE']


Task,One vs Rest,One vs Rest FLIGHTED,Two vs Rest,Two vs Rest FLIGHTED,Three vs Rest,Three vs Rest FLIGHTED,Low vs High,Low vs High FLIGHTED
Model,,,,,,,,
Linear,1.4706 ± 0.0127,12.0096 ± 0.0709,1.5174 ± 0.0020,9.8847 ± 0.0342,1.5172 ± 0.0049,8.1622 ± 0.0122,4.6324 ± 0.0126,7.5193 ± 0.1734
TAPE,1.5062 ± 0.0090,11.8502 ± 0.2601,1.9685 ± 0.1161,7.1776 ± 0.3339,0.9591 ± 0.0295,3.6993 ± 0.0375,3.5252 ± 0.0394,2.4557 ± 0.1641
ESM-1b,1.4898 ± 0.0004,11.2226 ± 0.0750,1.4427 ± 0.0107,5.7642 ± 0.2590,0.9484 ± 0.0104,3.2003 ± 0.0183,3.5314 ± 0.0218,2.5016 ± 0.1014
ESM-1v,1.5132 ± 0.0016,14.7296 ± 0.0125,1.5646 ± 0.0153,6.7795 ± 0.2354,0.9261 ± 0.0185,3.4494 ± 0.0882,3.4070 ± 0.0201,1.7771 ± 0.0768
ESM-2 (8M),1.5169 ± 0.0177,11.6852 ± 0.0226,1.4388 ± 0.0081,7.6910 ± 0.1750,1.0639 ± 0.0142,4.3834 ± 0.0081,3.6068 ± 0.0302,3.4855 ± 0.0535
ESM-2 (35M),1.5218 ± 0.0218,11.5808 ± 0.0899,1.4034 ± 0.0103,8.2036 ± 0.0373,0.9534 ± 0.0022,3.8483 ± 0.0362,3.5095 ± 0.0738,2.9552 ± 0.0789
ESM-2 (150M),1.5129 ± 0.0210,11.5738 ± 0.1378,1.4099 ± 0.0057,8.5423 ± 0.0238,0.9726 ± 0.0052,4.3019 ± 0.1824,3.5763 ± 0.0271,3.0476 ± 0.0451
ESM-2 (650M),1.4963 ± 0.0024,11.5768 ± 0.2312,1.4285 ± 0.0288,5.5143 ± 0.2098,0.7834 ± 0.0036,2.9651 ± 0.0779,3.4553 ± 0.0089,2.3014 ± 0.0535
ESM-2,1.9206 ± 0.0334,12.0280 ± 0.1538,1.3472 ± 0.0071,5.0572 ± 0.3065,0.8850 ± 0.0038,2.8603 ± 0.0090,3.4237 ± 0.0502,2.1913 ± 0.0138
